# Подготовка датасет

Этот скрипт поможет подготовить Excel файл с данными о погрузке одного из типов грузов в период с 2018 по август (включительно 2024 года). Оригинальный файл был загружен с сайта Ростата.

Датасет предполагается использовать для решения задач по прогнозированию.

In [1]:
#читаем файл с данными
import pandas as pd

# сохраним в этой переменной данные по всем грузам
data={}

#цикл с 2018 года по 2024 включительно 
for y in range(2018,2025):
    ys=str(y) # преобразуем год в строку
    # читаем файл листом номером года
    df=pd.read_excel("Pogruzka_08-2024.xlsx", sheet_name=ys)
    df.rename(columns={'Unnamed: 0':'Груз'}, inplace=True)
    df.drop(index=[1], inplace=True)

    sheet={}
    for s in df.index:
        name=df.loc[s,'Груз']
        sheet[name]={}
        for c in df.columns[1:]:
            sheet[name][c]=df.loc[s,c]
    data[y]=sheet

print('Выберите тип груза из списка:')
list(sheet.keys())

Выберите тип груза из списка:


['Грузы - всего, тысяч тонн',
 'каменный уголь',
 'кокс',
 'руда железная и марганцевая',
 'руды цветных металлов и серное сырье',
 'черные металлы',
 'лом черных металлов',
 'химические и минеральные удобрения',
 'строительные грузы',
 'цемент',
 'лесные грузы',
 'зерно и продукты перемола',
 'комбикорма',
 'импортные грузы',
 'рыба']

In [2]:
#скопируйте в качестве зачения переменной название груза в кавычках
gruz_name='комбикорма'

import calendar

def last_day_of_month(year, month):
    return calendar.monthrange(year, month)[1]

def convert_to_date(year, month):
    last_day = last_day_of_month(year, month)
    return pd.to_datetime(f"{year}-{month:02d}-{last_day:02d}")

# формируем датасет по одному грузу
one_item=[]

# собираем данные с 2018 года по 2024 включительно
for year in range(2018,2025):
    month=1 #номер стартового месяц и цикл по месяцам
    for one in data[year][gruz_name]:
        if 'год' in one: continue # не сохраняем стобец итого
        # собираем данные в список словарей
        one_item.append({'date':convert_to_date(year, month),  'value':data[year][gruz_name][one]})
        month+=1
df=pd.DataFrame(one_item)
df.dropna(inplace=True)
df.set_index('date', inplace=True)

#записываем результат в файл timeseries.xlsx
df.to_excel('timeseries.xlsx')